In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import StandardScaler

In [2]:
# Fog Data
fogData = xr.open_dataset('../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
y=fg.fogdata.sum(dim='stations').sel(time=slice('1980-12-1','2018-1-31'))
y=y/25.0
y

<xarray.DataArray 'fogdata' (time: 2356)>
array([0.12, 0.12, 0.12, ..., 0.64, 0.28, 0.16])
Coordinates:
  * time     (time) datetime64[ns] 1980-12-01 1980-12-02 ... 2018-01-31

In [3]:
AOData = xr.open_dataset('../../../Data/AO_EU/AO_Calculated.nc')
AO5D=AOData.AO.rolling(time=5).mean()
aoTS=AO5D
ao=aoTS[((aoTS.time.dt.month>11) | (aoTS.time.dt.month<2)) & (aoTS.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'))


In [4]:
xr.corr(y,ao,dim='time')

np.corrcoef(y,ao)

array([[ 1.       , -0.0227776],
       [-0.0227776,  1.       ]])

In [5]:
EUData = xr.open_dataset('../../../Data/AO_EU/EU.nc')
EU5D=EUData.EUVal.rolling(time=5).mean()
euTS=EU5D
eu=euTS[((euTS.time.dt.month>11) | (euTS.time.dt.month<2)) & (euTS.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'))

np.corrcoef(y,eu)


array([[ 1.       , -0.0116822],
       [-0.0116822,  1.       ]])

In [6]:
dataset=xr.open_dataset('VarDates.nc')
dataset
#y

<xarray.Dataset>
Dimensions:  (ncl0: 2480, ncl1: 36, ncl2: 2480)
Dimensions without coordinates: ncl0, ncl1, ncl2
Data variables:
    X        (ncl0, ncl1) float32 ...
    Y        (ncl2) float32 ...

In [7]:
varList=["t2mVal","wind10Val","rh2mVal","invVal","wVal",
         "v_shearVal","u_shearVal","t2mTendVal","mslTendVal",
         "wTendVal","qadvTendVal","z500Val","t2mVal1D","wind10Val1D",
         "rh2mVal1D","invVal1D","wVal1D","v_shearVal1D","u_shearVal1D",
         "t2mTendVal1D","mslTendVal1D","wTendVal1D","qadvTendVal1D","z500Val1D","t2mVal2D",
         "wind10Val2D","rh2mVal2D","invVal2D","wVal2D","v_shearVal2D",
         "u_shearVal2D","t2mTendVal2D","mslTendVal2D","wTendVal2D",
         "qadvTendVal2D","z500Val2D"]

In [8]:
#ff=dataset.Y
#ff.assign_coords(ncl2=y.time)

ff=xr.Dataset({'ff': (('time'), dataset.Y.values)}, coords={'time': y.time})
#ff

Xs=xr.Dataset({'x': (('time','vars'), dataset.X.values)}, coords={'time': y.time,'vars': varList})
Xs

ValueError: conflicting sizes for dimension 'time': length 2356 on 'time' and length 2480 on 'ff'

In [ ]:
CC=xr.corr(ff.ff,Xs.x,dim='time')
for i in CC:
    print(i.values)

